In [1]:
import pickle

datapath = "/Users/ufuk/Research/AIMS/Project Repo/eso/data/Saved_Data/preprocessed"

with open(datapath + "/X.pkl", "rb") as f:
    X = pickle.load(f)
with open(datapath + "/Y.pkl", "rb") as f:
    Y = pickle.load(f)

NameError: name 'pickle' is not defined

In [3]:
X.shape, Y.shape

((2424, 128, 76), (2424,))

In [4]:
import torch.nn as nn
import numpy as np
import torch


def get_conv_output_dim(layer, input_dim):
    """Calculate output dimension of a CNN layer"""
    kernel_size = layer.kernel_size
    stride = layer.stride
    padding = layer.padding
    dilation = layer.dilation

    input_channels, input_height, input_width = input_dim

    output_channels = layer.out_channels
    output_height = (
        input_height + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1
    ) / stride[0] + 1
    output_width = (
        input_width + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1
    ) / stride[1] + 1

    return (output_channels, int(output_height), int(output_width))


class BaseCNN(nn.Module):
    def __init__(self, input_shape):
        super(BaseCNN, self).__init__()
        self.input_shape = input_shape

        n_channels, height, width = input_shape

        self.conv = nn.Sequential(
            nn.Conv2d(n_channels, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(1),
        )

        self.cnn_output_dim = self._calc_cnn_output_dim()

        self.fully_connected = nn.Sequential(
            nn.Linear(np.prod(self.cnn_output_dim), 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 2),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        if len(x.shape) == 3:
            x = x.unsqueeze(1)
        x = self.conv(x)
        x = x.view(-1, np.prod(self.cnn_output_dim))
        x = self.fully_connected(x)
        return x

    def _calc_cnn_output_dim(self):
        """Calculate output dimension of the CNN part of the network"""

        output_dim = get_conv_output_dim(self.conv[0], self.input_shape)
        for layer in self.conv[1:]:
            # Check if layer is a convolutional layer
            if isinstance(layer, nn.Conv2d):
                output_dim = get_conv_output_dim(layer, output_dim)

        return output_dim


class Model:
    """Model class."""

    def __init__(self, input_shape):
        self.cnn = BaseCNN(input_shape)
        # self.logger.info("Initializing Model...")
        # Get Device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def get_number_of_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    def train_model(
        self, n_epochs, train_loader, optimizer, criterion, save_path=None, verbose=True
    ):
        print("Training")

        self.cnn.to(self.device)
        train_losses = []
        val_losses = []
        val_accs = []
        best_val_acc = 0

        for epoch in range(n_epochs):
            print("Epoch: ", epoch)
            for batch_inputs, batch_targets in train_loader:
                batch_inputs, batch_targets = (
                    batch_inputs.to(self.device),
                    batch_targets.to(self.device),
                )
                # Reset gradients
                optimizer.zero_grad()
                # Forward pass
                batch_preds = self.cnn.forward(batch_inputs)
                # Compute loss
                loss = criterion(batch_preds, batch_targets)
                # Backward and optimize
                loss.backward()
                optimizer.step()
                train_losses.append(loss.item())
        return train_losses

    def __call__(self, x):
        return self.cnn(x)

    def evaluate(self, loader, criterion):
        self.cnn.eval()
        with torch.no_grad():
            total_loss = 0
            correct = 0
            for batch_inputs, batch_targets in loader:
                batch_inputs, batch_targets = (
                    batch_inputs.to(self.device),
                    batch_targets.to(self.device),
                )
                batch_preds = self.cnn.forward(batch_inputs)

                total_loss += criterion(batch_preds, batch_targets).item()
                correct += (batch_preds.argmax(dim=1) == batch_targets).sum().item()
            average_loss = total_loss / len(loader)
            accuracy = correct / len(loader.dataset)
        self.cnn.train()
        return average_loss, accuracy

In [19]:
X.shape

(2424, 128, 76)

In [20]:
from model import Model

In [21]:
m = Model(input_shape=(1, X.shape[1], X.shape[2]))
test = torch.from_numpy(X[0]).float()
test = test.unsqueeze(0)
print(test.shape)
m(test)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


# Encode class labels to integers using LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

label_classes = dict(
    zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))
)

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.Tensor(X_train)
X_test_tensor = torch.Tensor(X_test)
y_train_tensor = torch.Tensor(y_train_encoded).long()
y_test_tensor = torch.Tensor(y_test_encoded).long()


# Create one-hot encodings for class labels
num_classes = len(label_encoder.classes_)
y_train_onehot = torch.zeros(len(y_train_encoded), num_classes)
y_train_onehot.scatter_(1, y_train_tensor.view(-1, 1), 1)


# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

torch.Size([1, 128, 76])


In [22]:
X_train.shape

(1939, 128, 76)

In [23]:
m.train(5, X_train=X_train, Y_train=y_train_encoded)

Training


RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[1, 32, 128, 76] to have 1 channels, but got 32 channels instead

TypeError: Model.train() missing 5 required positional arguments: 'n_epochs', 'X_train', 'Y_train', 'optimizer', and 'criterion'

In [29]:
m = Model(input_shape=(1, X.shape[1], X.shape[2]))
test = torch.from_numpy(X[0]).float()
test = test.unsqueeze(0)
print(test.shape)
m(test)

torch.Size([1, 128, 76])


tensor([[0.4578, 0.5422]], grad_fn=<SoftmaxBackward0>)

In [30]:
m = Model(input_shape=(1, X.shape[1], X.shape[2]))
test = torch.Tensor(X[0])
test = test.unsqueeze(0)
print(test.shape)
m(test)

torch.Size([1, 128, 76])


tensor([[0.5028, 0.4972]], grad_fn=<SoftmaxBackward0>)

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


# Encode class labels to integers using LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

label_classes = dict(
    zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))
)

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.Tensor(X_train)
X_test_tensor = torch.Tensor(X_test)
y_train_tensor = torch.Tensor(y_train_encoded).long()
y_test_tensor = torch.Tensor(y_test_encoded).long()


# Create one-hot encodings for class labels
num_classes = len(label_encoder.classes_)
y_train_onehot = torch.zeros(len(y_train_encoded), num_classes)
y_train_onehot.scatter_(1, y_train_tensor.view(-1, 1), 1)


# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [33]:
m.train_model(
    n_epochs=3,
    train_loader=train_loader,
    optimizer=torch.optim.Adam(m.cnn.parameters(), lr=0.001),
    criterion=nn.CrossEntropyLoss(),
)

Training
Epoch:  0
Epoch:  1
Epoch:  2


[0.6969032883644104,
 0.46951162815093994,
 0.5320115685462952,
 0.5007615685462952,
 0.5632616281509399,
 0.5007616281509399,
 0.34451165795326233,
 0.5945116281509399,
 0.5320115685462952,
 0.7195115685462952,
 0.43826159834861755,
 0.31326165795326233,
 0.43826162815093994,
 0.46951165795326233,
 0.40701165795326233,
 0.5007616281509399,
 0.5007616281509399,
 0.5320115685462952,
 0.37576159834861755,
 0.43826165795326233,
 0.5632616877555847,
 0.6882616877555847,
 0.46951159834861755,
 0.5007616281509399,
 0.5007616281509399,
 0.46951162815093994,
 0.5945116281509399,
 0.46951162815093994,
 0.46951162815093994,
 0.5007615685462952,
 0.5007616281509399,
 0.43826162815093994,
 0.6257616877555847,
 0.5320116281509399,
 0.46951159834861755,
 0.46951159834861755,
 0.43826165795326233,
 0.43826159834861755,
 0.5007615685462952,
 0.5320115685462952,
 0.5320116281509399,
 0.46951165795326233,
 0.5320115685462952,
 0.5320116281509399,
 0.43826162815093994,
 0.6257616877555847,
 0.50076162815

In [22]:
m.evaluate(test_loader, nn.CrossEntropyLoss())

(0.5402147453278303, 0.7938144329896907)

In [37]:
m.get_number_of_parameters()

18692642

In [114]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)

cnn.eval()
correct = 0
total = 0

accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
confusion_matrices = []

with torch.no_grad():
    for batch_inputs, batch_labels in test_loader:
        batch_inputs = batch_inputs.unsqueeze(1)
        outputs = cnn(batch_inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += len(batch_inputs)
        correct += (predicted == batch_labels).sum().item()

        accuracy_scores.append(accuracy_score(batch_labels, predicted))
        precision_scores.append(precision_score(batch_labels, predicted))
        recall_scores.append(recall_score(batch_labels, predicted))
        f1_scores.append(f1_score(batch_labels, predicted))
        confusion_matrices.append(confusion_matrix(batch_labels, predicted))


accuracy = correct / total
print(f"Test Accuracy: {100 * accuracy:.2f}%")

Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125, 73)
Actual shape: torch.Size([32, 32, 125, 73])
Predicted shape: (32, 125